In [12]:
import pandas as pd
from sys import exit
from os.path import isdir, isfile
from os import listdir, mkdir
import Bio.SeqIO as IO
import numpy as np

def Parse_HMM_Alignments(align):
    OP = []
    counts = 0
    for a in align.split("//\n"):
        o = a.split("\n")
        align_flag = False
        ctr = 0
        while ctr < len(o):
            if o[ctr].lstrip().startswith("Query:"):
                query = o[ctr].lstrip().replace("Query:","").lstrip().split(' ')[0]
                ctr += 1
            elif o[ctr].lstrip().startswith('>>'):
                print(o[ctr])
                subject = o[ctr].lstrip().replace(">>","").rstrip()
                aligns = []
                if "[No individual domains that satisfy reporting thresholds (although complete target did)]" in o[ctr+1]:
                    ctr += 2
                    continue
                ctr += 2
                while ("Alignments for each domain" not in o[ctr]) and ("Internal pipeline statistics" not in o[ctr]):
                    if o[ctr].lstrip().startswith("--"):
                        ctr += 1
                        continue
                    values = o[ctr].lstrip().replace("[.","").replace(".]","").replace("..","").replace("[]","").split(' ')
                    values = [t for t in values if t != '']
                    ctr += 1
                    header = ['No', 'Score','bias','c-Evalue', 'i-Evalue','hmmfrom','hmmto',
                              'alifrom','alito','envfrom', 'envto', 'acc']
                    print(values)
                    values = [float(v) for v in values if v != '!' and v != '?']
                    d = dict(zip(header, values))
                    d['Query'] = query
                    d['Subject'] = subject
                    d['Subject_Alignment'] = ''
                    d['Alignment_Description'] = ''
                    d['Query_Alignment'] = ''
                    aligns.append(d)
                
                if o[ctr].lstrip().startswith('Alignments for each domain') :
                    cur = 0
                    align_flag = True
                    ctr += 2
            
            if(align_flag):
                if (o[ctr].lstrip().startswith("==")):
                    cur += 1
                    ctr += 1
                tgt = o[ctr].lstrip().split(' ')
                tgt = [t for t in tgt if t != '']
                mat = o[ctr+1].lstrip().replace(" ","S")
                qry = o[ctr+2].lstrip().split(' ')
                qry = [t for t in qry if t != '']
                aligns[cur]['Subject_Alignment'] += tgt[2]
                aligns[cur]['Query_Alignment']  += qry[2]
                aligns[cur]['Alignment_Description']  += 'S'*(len(mat)-len(tgt[2]))+mat
                ctr += 4

                if o[ctr].startswith('Internal') or o[ctr].lstrip().startswith('>>') :
                    counts += 1
                    OP += aligns
                    align_flag = False
                if o[ctr].startswith('Internal'):
                    break
            else:
                ctr += 1
            
            
    df_OP = pd.DataFrame(OP)
    return df_OP

def Calculate_Untagged_Gene_Lengths(filepath):
    d = {}
    record_dict = IO.to_dict(IO.parse(filepath, "fasta"))
    for key in record_dict.items():
        d[key[0]] = len(key[1].seq)
    df_lengths = pd.DataFrame(data = {'Query':list(d.keys()), 'Length':list(d.values())})
    return df_lengths

def Load_HMMER_Outputs(filepath, df_lengths):
    df = pd.read_csv(filepath)
    del df['Unnamed: 0']
   
    return df

def Return_Breadth_of_Coverage(grp):
    grp = grp.sort_values(by = 'alifrom')
    starts = grp['alifrom'].tolist()
    ends = grp['alito'].tolist()
    breadth = 0
    for i in range(1, len(starts)):
        prev_end = ends[i-1]
        curr_starts = starts[i]
        dist = curr_starts - prev_end
        if dist > 0:
            breadth += dist
    breadth = (starts[0] - 0) + breadth + (grp.iloc[0]['Length'] - ends[-1])
    breadth = breadth/grp.iloc[0]['Length'] * 100.0
    norm = np.sum(grp['alito'] - grp['alifrom'])
    score = np.sum((grp['alito'] - grp['alifrom'])*grp['Score']/(norm))
    cevlaue = np.sum((grp['alito'] - grp['alifrom'])*grp['c-Evalue']/norm)
    ievlaue = np.sum((grp['alito'] - grp['alifrom'])*grp['i-Evalue']/norm)
    
    return pd.Series({'breadth':breadth, 'Counts':len(grp), 'Avg. Score':score, 
                      'Avg. c-Evalue':cevlaue, 'Avg. i-Evalue':ievlaue})


In [13]:
filepath = '/Users/harihara/Mount/CMSC829A/Data/Untagged_Accessory_Genes_Proteins_HMM_Alignment/out/'
untagged_gene_seq = '/Users/harihara/Mount/CMSC829A/Data/Untagged_Accessory_Genes_Proteins/'

In [14]:
f = GCF_001007085.1_Wien3_genomic.out
lines = open(filepath+f).readlines()
out = ''
for line in lines:
    if line.startswith('#') or line.startswith('--') or line == "\n":
        continue
    else:
        out += line
df_HMMER = Parse_HMM_Alignments(out)
df_lengths = Calculate_Untagged_Gene_Lengths(untagged_gene_seq+f.replace(".out",".faa"))
df_HMMER = pd.merge(df_HMMER, df_lengths, on = ['Query'], how = 'left')
df_HMMER = df_HMMER[df_HMMER['Score'] >= 0]
df_HMMER_grp = df_HMMER.groupby(['Query','Subject']).apply(Return_Breadth_of_Coverage)
df_seqs = df_HMMER[['Query','Subject','Length','Subject_Alignment','Alignment_Description','Query_Alignment']]
df_seqs = df_seqs.set_index(['Query', 'Subject'])
df_HMMER_grp = df_HMMER_grp.join(df_seqs, how = 'left')
df_HMMER_grp = df_HMMER_grp.reset_index()
min_breadth = 60
df_HMMER_filter = df_HMMER_grp[df_HMMER_grp['breadth'] >= min_breadth]
df_HMMER_filter = df_HMMER_filter.loc[df_HMMER_filter.groupby('Query')['Avg. Score'].idxmax()]
df_HMMER_filter.to_csv(tophit_dir+f, sep = '\t')
print(f)

>> xseB  
['1', '?', '-0.0', '0.1', '0.043', '1.3e+02', '48', '63', '198', '213', '179', '224', '0.70']
['2', '?', '1.5', '0.2', '0.014', '42', '47', '68', '220', '241', '189', '249', '0.74']
['3', '?', '7.9', '0.9', '0.00015', '0.43', '12', '61', '241', '290', '238', '310', '0.90']
['4', '?', '-0.9', '0.1', '0.081', '2.4e+02', '47', '66', '296', '315', '294', '341', '0.71']
>> dinG  
['1', '?', '-2.9', '0.1', '0.16', '2.4e+02', '289', '320', '81', '112', '72', '144', '0.59']
['2', '?', '10.7', '4.6', '1.2e-05', '0.018', '265', '401', '385', '535', '378', '546', '0.75']
['3', '?', '-0.9', '0.2', '0.038', '56', '322', '364', '664', '706', '580', '746', '0.57']
>> xseB  
['1', '?', '0.6', '0.1', '0.057', '83', '41', '68', '61', '88', '57', '122', '0.70']
['2', '?', '-3.2', '0.0', '0.86', '1.3e+03', '8', '22', '225', '239', '223', '241', '0.77']
['3', '?', '-2.2', '0.0', '0.42', '6.1e+02', '49', '63', '277', '291', '275', '294', '0.82']
['4', '?', '4.0', '0.8', '0.0047', '6.9', '32', '71'

>> ccmA  
['1', '!', '104.1', '0.4', '3.1e-32', '1.2e-30', '8', '194', '14', '208', '4', '220', '0.84']
>> phnT  
['1', '!', '103.9', '0.0', '3.3e-32', '1.3e-30', '29', '230', '19', '223', '9', '226', '0.85']
>> tauB  
['1', '!', '103.7', '0.1', '4.2e-32', '1.7e-30', '7', '189', '14', '204', '5', '219', '0.84']
>> hisP  
['1', '!', '99.5', '0.1', '6.9e-31', '2.7e-29', '12', '222', '15', '214', '5', '224', '0.87']
>> ugpC  
['1', '!', '101.6', '0.0', '1.8e-31', '7.2e-30', '7', '194', '12', '204', '4', '221', '0.81']
>> modB  
['1', '!', '99.2', '0.5', '5.6e-31', '2.3e-29', '269', '473', '14', '221', '4', '225', '0.85']
>> thiQ  
['1', '!', '98.5', '0.4', '1.4e-30', '5.6e-29', '17', '209', '26', '222', '9', '226', '0.83']
>> proV  
['1', '!', '98.4', '0.1', '1.4e-30', '5.8e-29', '36', '241', '15', '219', '1', '223', '0.84']
>> gltL  
['1', '!', '98.1', '0.0', '1.7e-30', '6.9e-29', '4', '205', '7', '213', '4', '224', '0.85']
>> potG  
['1', '!', '97.8', '0.0', '2.2e-30', '8.9e-29', '18', 

>> oppF  
['1', '!', '76.2', '0.0', '7.4e-24', '3.1e-22', '38', '265', '21', '241', '8', '245', '0.77']
>> msbA  
['1', '!', '28.4', '0.0', '1.8e-09', '7.6e-08', '346', '400', '8', '62', '2', '66', '0.91']
['2', '!', '45.9', '0.0', '9.4e-15', '4e-13', '480', '541', '140', '201', '127', '216', '0.91']
>> btuD  
['1', '!', '73.6', '0.2', '5.4e-23', '2.3e-21', '17', '213', '23', '216', '19', '230', '0.77']
>> urtD  
['1', '!', '23.6', '0.0', '1.2e-07', '5.1e-06', '26', '69', '21', '64', '10', '86', '0.90']
['2', '!', '48.4', '0.0', '3.2e-15', '1.3e-13', '124', '226', '112', '214', '93', '227', '0.91']
>> choV  
['1', '!', '67.7', '0.0', '3.6e-21', '1.5e-19', '43', '242', '22', '214', '20', '225', '0.85']
>> potG  
['1', '!', '69.5', '0.0', '8.2e-22', '3.5e-20', '31', '225', '18', '215', '4', '219', '0.77']
>> ettA  
['1', '!', '30.0', '0.1', '7.2e-10', '3e-08', '337', '381', '20', '64', '4', '75', '0.84']
['2', '!', '38.8', '0.1', '1.6e-12', '6.7e-11', '143', '224', '121', '205', '107', '

>> atpA  
['1', '!', '127.5', '0.0', '4.1e-40', '9.9e-38', '25', '383', '17', '370', '2', '403', '0.85']
>> rho  
['1', '!', '37.5', '0.0', '6.9e-13', '1.7e-10', '163', '321', '146', '307', '113', '385', '0.80']
>> oppF  
['1', '!', '15.7', '0.0', '3.2e-06', '0.00079', '35', '133', '144', '244', '137', '266', '0.84']
>> rarA  
['1', '!', '11.8', '0.0', '4.2e-05', '0.01', '49', '112', '156', '218', '141', '227', '0.85']
>> cydD  
['1', '?', '11.1', '0.3', '7.1e-05', '0.017', '361', '388', '151', '178', '142', '224', '0.84']
>> phnC  
['1', '?', '10.5', '0.0', '0.00015', '0.037', '18', '87', '145', '216', '133', '278', '0.76']
>> thiQ  
['1', '?', '10.5', '0.1', '0.00019', '0.046', '20', '52', '152', '184', '146', '207', '0.82']
>> fliH  
['1', '!', '27.0', '0.1', '2.9e-10', '4.2e-07', '107', '242', '35', '173', '31', '177', '0.90']
>> sctL  
['1', '!', '13.0', '0.0', '7.1e-06', '0.01', '117', '202', '93', '176', '50', '178', '0.76']
>> fliG  
['1', '?', '-3.0', '0.0', '0.8', '5.8e+02', 

NameError: name 'tophit_dir' is not defined